# Visualisation of the BERT Attention using `bertviz` 

- Please run the `bert_model.ipynb` before running this nodebook.

## Load the Tokenizer model

In [ ]:
from transformers import BertTokenizer, TFBertForSequenceClassification, BertConfig
import os
import tensorflow as tf

tokenizer_model_path = './models/BERT/'
bert_model_path = './models/BERT/'

# Load the tokenizer
if os.path.exists(tokenizer_model_path):
    print("Loading the existing tokenizer...")
    tokenizer = BertTokenizer.from_pretrained(tokenizer_model_path)
else:
    print("No existing tokenizer found... Please run the `bert_model.ipynb` notebook first.")


## Load the BERT model

In [ ]:
config = BertConfig.from_pretrained('bert-base-uncased', output_attentions=True)

optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5, epsilon=1e-08)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')

# Load the BERT model
if os.path.exists(bert_model_path):
    print("Loading the existing model...")
    model_bert = TFBertForSequenceClassification.from_pretrained(bert_model_path, config=config)
    model_bert.compile(optimizer=optimizer, loss=loss, metrics=[metric])
else:
    print("No existing model found... Please run the `bert_model.ipynb` notebook first.")
    

## Visualise the attention score for the sentence `love it, a great upgrade from the original.  I've had mine for a couple of years`

In [ ]:
from bertviz import head_view
import torch

sentence = 'love it, a great upgrade from the original.  I\'ve had mine for a couple of years'
encoded_dict = tokenizer.encode_plus(
    sentence,
    add_special_tokens = True,
    max_length = 50,
    pad_to_max_length = True,
    return_attention_mask = True,
    return_tensors = 'tf',
)

outputs = model_bert(input_ids=encoded_dict['input_ids'], attention_mask=encoded_dict['attention_mask'], return_dict=True)

attention = outputs[-1]
attention = [torch.from_numpy(layer_attn.numpy()) for layer_attn in attention]
tokens = tokenizer.convert_ids_to_tokens(encoded_dict['input_ids'][0])  

In [ ]:
attention

In [ ]:
head_view(attention, tokens)